<a href="https://colab.research.google.com/github/rafiafsan/LSTM-Long-Short-Term-Memory-/blob/main/10_fold_cross_validation_EEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [2]:
!unzip /content/archive.zip -d /content

Archive:  /content/archive.zip
  inflating: /content/Bonn Univeristy Dataset/F/F001.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F002.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F003.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F004.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F005.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F006.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F007.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F008.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F009.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F010.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F011.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F012.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F013.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F014.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F015.txt  
  inflating: /content/Bonn Univeristy Dataset/F/F016.txt  
  inflating: /content/Bon

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [5]:
extract_path = '/content/Bonn Univeristy Dataset'

class_folders = ['Z', 'O', 'N', 'F', 'S']
class_labels = {'Z': 0, 'O': 1, 'N': 2, 'F': 3, 'S': 4}

eeg_data = []
eeg_labels = []

for class_folder in class_folders:
    class_path = os.path.join(extract_path, class_folder)
    if not os.path.exists(class_path):
        print(f"Folder not found: {class_folder}")
        continue
    for filename in os.listdir(class_path):
        if filename.lower().endswith('.txt'):
            file_path = os.path.join(class_path, filename)
            data = np.loadtxt(file_path)
            eeg_data.append(data[:4096])  # Trim/pad if needed
            eeg_labels.append(class_labels[class_folder])

eeg_data = np.array(eeg_data)
eeg_labels = np.array(eeg_labels)
print("Data shape:", eeg_data.shape)
print("Labels shape:", eeg_labels.shape)

Data shape: (500, 4096)
Labels shape: (500,)


In [6]:
eeg_data_reshaped = eeg_data.reshape(eeg_data.shape[0], 2048, 2)
print("Reshaped EEG data:", eeg_data_reshaped.shape)

Reshaped EEG data: (500, 2048, 2)


In [8]:
from tensorflow.keras.utils import to_categorical

In [9]:
eeg_labels_cat = to_categorical(eeg_labels, num_classes=5)

In [10]:
from sklearn.model_selection import StratifiedKFold

In [11]:
# Step 2: Set up cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GlobalAveragePooling1D

In [14]:
fold = 1
accuracies = []

for train_index, test_index in skf.split(eeg_data_reshaped, eeg_labels):
    print(f"Running Fold {fold}...")

    X_train, X_test = eeg_data_reshaped[train_index], eeg_data_reshaped[test_index]
    y_train, y_test = eeg_labels_cat[train_index], eeg_labels_cat[test_index]

    # Step 3: Build LSTM model
    model = Sequential()
    model.add(LSTM(64, input_shape=(2048, 2)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(5, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Step 4: Train model
    model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

    # Step 5: Evaluate
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Fold {fold} Accuracy: {accuracy:.4f}")
    accuracies.append(accuracy)

    fold += 1

# Step 6: Average Accuracy
print(f"\nAverage Accuracy over 10 folds: {np.mean(accuracies):.4f}")

Running Fold 1...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Fold 1 Accuracy: 0.3600
Running Fold 2...
Fold 2 Accuracy: 0.3000
Running Fold 3...
Fold 3 Accuracy: 0.3800
Running Fold 4...
Fold 4 Accuracy: 0.4200
Running Fold 5...
Fold 5 Accuracy: 0.4200
Running Fold 6...
Fold 6 Accuracy: 0.4600
Running Fold 7...
Fold 7 Accuracy: 0.3800
Running Fold 8...
Fold 8 Accuracy: 0.4400
Running Fold 9...
Fold 9 Accuracy: 0.3800
Running Fold 10...
Fold 10 Accuracy: 0.4200

Average Accuracy over 10 folds: 0.3960
